# Workshop 4 Exercise 2 Time Series Classification

link to the LSTM slides

https://github.com/jojker/PML_Workshops/blob/master/Summer%202019/Day%204%20-%20Goal%203%20-%20Identifying%2C%20Labeling%2C%20and%20Classifying/Ex%202%20-%20Time%20Series%20Classification/LSTM_notes.pptx

In [0]:
# Run this file immediately to extract a large zip file from a drive account
# ~ 10 minutes

# method to get a large file from google drive 

fileid='1Te8hQcCINvabe7WttGdpE1ojQtqogI89'
filename='train_simplified.zip'

## WGET ##
!wget --save-cookies cookies.txt 'https://docs.google.com/uc?export=download&id='$fileid -O- \
     | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1/p' > confirm.txt

!wget --load-cookies cookies.txt -O $filename \
     'https://docs.google.com/uc?export=download&id='$fileid'&confirm='$(<confirm.txt)

## CURL ##
!curl -L -c cookies.txt 'https://docs.google.com/uc?export=download&id='$fileid \
     | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1/p' > confirm.txt

!curl -L -b cookies.txt -o $filename \
     'https://docs.google.com/uc?export=download&id='$fileid'&confirm='$(<confirm.txt)

!rm -f confirm.txt cookies.txt

# unzip the file
!unzip train_simplified.zip

In [0]:
# get the test set from github
fileid='1VOzqQDVMe5Jge2eD6RhmOnFsnll_A--b'
filename='test_simplified.csv'

## WGET ##
!wget --save-cookies cookies.txt 'https://docs.google.com/uc?export=download&id='$fileid -O- \
     | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1/p' > confirm.txt

!wget --load-cookies cookies.txt -O $filename \
     'https://docs.google.com/uc?export=download&id='$fileid'&confirm='$(<confirm.txt)

## CURL ##
!curl -L -c cookies.txt 'https://docs.google.com/uc?export=download&id='$fileid \
     | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1/p' > confirm.txt

!curl -L -b cookies.txt -o $filename \
     'https://docs.google.com/uc?export=download&id='$fileid'&confirm='$(<confirm.txt)

!rm -f confirm.txt cookies.txt

In [0]:
# get pretrained weights from a google drive account
fileid='16reUvzJ0eYpNsht_6PyUJG0heVJtKyQu'
filename='stroke_lstm_model_weights.best.hdf5'

## WGET ##
!wget --save-cookies cookies.txt 'https://docs.google.com/uc?export=download&id='$fileid -O- \
     | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1/p' > confirm.txt

!wget --load-cookies cookies.txt -O $filename \
     'https://docs.google.com/uc?export=download&id='$fileid'&confirm='$(<confirm.txt)

## CURL ##
!curl -L -c cookies.txt 'https://docs.google.com/uc?export=download&id='$fileid \
     | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1/p' > confirm.txt

!curl -L -b cookies.txt -o $filename \
     'https://docs.google.com/uc?export=download&id='$fileid'&confirm='$(<confirm.txt)

!rm -f confirm.txt cookies.txt

# Keras tutorial using TensorFlow backend
The majority of the notebook was created by Kevin Mader for the "Quick, Draw! Doodle Recognition Challenge" on Kaggle 

https://www.kaggle.com/c/quickdraw-doodle-recognition

https://www.kaggle.com/kmader/quickdraw-baseline-lstm-reading-and-submission/notebook

Kaggle description of the challenge:

"Quick, Draw!" was released as an experimental game to educate the public in a playful way about how AI works. The game prompts users to draw an image depicting a certain category, such as ”banana,” “table,” etc. The game generated more than 1B drawings, of which a subset was publicly released as the basis for this competition’s training set. That subset contains 50M drawings encompassing 340 label categories.

Sounds fun, right? Here's the challenge: since the training data comes from the game itself, drawings can be incomplete or may not match the label. You’ll need to build a recognizer that can effectively learn from this noisy data and perform well on a manually-labeled test set from a different distribution.

Your task is to build a better classifier for the existing Quick, Draw! dataset. By advancing models on this dataset, Kagglers can improve pattern recognition solutions more broadly. This will have an immediate impact on handwriting recognition and its robust applications in areas including OCR (Optical Character Recognition), ASR (Automatic Speech Recognition) & NLP (Natural Language Processing).

<img src=https://hexacoto.files.wordpress.com/2016/11/screen-shot-2016-11-17-at-5-22-04-pm.png width="500">

# Kaggle description of the dataset
The Quick Draw Dataset is a collection of millions of drawings across 300+ categories, contributed by players of Quick, Draw! The drawings were captured as timestamped vectors, tagged with metadata including what the player was asked to draw and in which country the player was located.

Two versions of the data are given. The raw data is the exact input recorded from the user drawing, while the simplified version removes unnecessary points from the vector information. (For example, a straight line may have been recorded with 8 points, but since you only need 2 points to uniquely identify a line, 6 points can be dropped.) The simplified files are much smaller and provide effectively the same information.

For this competition, you may use the raw files, the simplified files, or both. You can find out more details about the drawing format on the quickdraw-dataset github page.

Your models should predict the correct "word" of the drawing. IMPORTANT: Some "words" are actually more than one word! The training data aligns to the Quick Draw dataset that that was previously released, and uses spaces to delimit multi-word labels. The Kaggle metric for this competition requires labels with no spaces, so you will need to adjust your label predictions to replace spaces with underscores. For example, "roller coaster" should be predicted as "roller_coaster". You may predict up to 3 guesses per drawing. See the Evaluation page for the correct format.

Here we will use the simplified dataset!

# Outline of the project
1. preprocess data using 1D convolutions
2. stack two LSTM (Long Short-Term Memory) neural networks
3. apply two dense neural network layers
4. classify shape
5. create submittable file for Kaggle competition

# CNN - Convolutional Neural Network
Images are represented as matrices of numbers. For greyscale images these numbers range from 0 to 255. Matrices are convolved to created a feature map. For example a 7x7 matrix convolved with a 3x3 matrix may look like

<img src=https://anhvnn.files.wordpress.com/2018/02/convolve.png? width="500">

# LSTM - Long Short-Term Memory
LSTM networks are units of a Recurrent Neural Network (RNN) commonly composed of four parts. 1) cell - remembers values over a certain time 2) input gate - controls the extent to which a new value flows into the cell 3) output gate -  controls the extent to which the cell value controls the output activation 4) forget gate - controlling the extent to which the value remains in the cell

# Model Parameters
Here we keep track of the relevant parameters for the data preprocessing, model construction and training

In [0]:
#batch_size = 32 # local machine will likely require a batch size much smaller than 2048
batch_size = 2048
STROKE_COUNT = 196
TRAIN_SAMPLES = 750
VALID_SAMPLES = 75
TEST_SAMPLES = 50

# check data storage on colab
!df -h

In [0]:
# Imports
%matplotlib inline

import os
import numpy as np
import matplotlib.pyplot as plt
from keras.utils.np_utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from keras.metrics import top_k_categorical_accuracy
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
from glob import glob

# garbage collection (automated memory management)
import gc
gc.enable()

# function for calculating the mean accuracy across k number of predictions for multiclass classification problems
def top_3_accuracy(x,y): return top_k_categorical_accuracy(x,y, 3)

# function for checking the number of available GPUs
def get_available_gpus():
    from tensorflow.python.client import device_lib
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

In [0]:
# add the location of the data to the path
working_dir = os.getcwd()

# location of test data file
test_path = os.path.join(working_dir, 'test_simplified.csv')

# location of pre-trained weights or location of place to save weights
weight_path = os.path.join(working_dir, "{}_weights.best.hdf5").format('stroke_lstm_model')
new_weight_path = os.path.join(working_dir, "{}_weights.new.hdf5").format('stroke_lstm_model')

print(working_dir)
print(test_path)
print(weight_path)

In [0]:
from ast import literal_eval

# glob here finds all the csv files containing the data to train on
ALL_TRAIN_PATHS = glob(os.path.join(working_dir, '*.csv'))

test_simplified_file = os.path.join(working_dir, 'test_simplified.csv')

# make sure the test file is not in the training file path
if test_simplified_file in ALL_TRAIN_PATHS: ALL_TRAIN_PATHS.remove(test_simplified_file)

print("number of csv files containing data: ", len(ALL_TRAIN_PATHS))

# column names of the csv files
COL_NAMES = ['countrycode', 'drawing', 'key_id', 'recognized', 'timestamp', 'word']

def _stack_it(raw_strokes):
    """preprocess the string and make 
    a standard Nx3 stroke vector"""
    stroke_vec = literal_eval(raw_strokes) # string->list
    # unwrap the list
    in_strokes = [(xi,yi,i)  
     for i,(x,y) in enumerate(stroke_vec) 
     for xi,yi in zip(x,y)]
    c_strokes = np.stack(in_strokes)
    # replace stroke id with 1 for continue, 2 for new
    c_strokes[:,2] = [1]+np.diff(c_strokes[:,2]).tolist()
    c_strokes[:,2] += 1 # since 0 is no stroke
    # pad the strokes with zeros
    return pad_sequences(c_strokes.swapaxes(0, 1), 
                         maxlen=STROKE_COUNT, 
                         padding='post').swapaxes(0, 1)
def read_batch(samples=5, 
               start_row=0,
               max_rows = 1000):
    """
    load and process the csv files
    this function is horribly inefficient but simple
    """
    out_df_list = []
    for c_path in ALL_TRAIN_PATHS:
        c_df = pd.read_csv(c_path, nrows=max_rows, skiprows=start_row)
        c_df.columns=COL_NAMES
        out_df_list += [c_df.sample(samples)[['drawing', 'word']]]
    full_df = pd.concat(out_df_list)
    full_df['drawing'] = full_df['drawing'].\
        map(_stack_it)
    
    return full_df

# Reading and Parsing
Since it is too much data (7GB, the full data set from kaggle is 23GB) to read in at once, we just take a portion of it for training, validation and hold-out testing. This should give us an idea about how well the model works, but leaves lots of room for improvement later

In [0]:
# take a portion of the data from the unzipped csv files to be training data, validation data, and testing data
train_args = dict(samples=TRAIN_SAMPLES, 
                  start_row=0, 
                  max_rows=int(TRAIN_SAMPLES*1.5))
valid_args = dict(samples=VALID_SAMPLES, 
                  start_row=train_args['max_rows']+1, 
                  max_rows=VALID_SAMPLES+25)
test_args = dict(samples=TEST_SAMPLES, 
                 start_row=valid_args['max_rows']+train_args['max_rows']+1, 
                 max_rows=TEST_SAMPLES+25)

# read in the data using the definitions in previous cell
train_df = read_batch(**train_args)
valid_df = read_batch(**valid_args)
test_df = read_batch(**test_args)

# encode labels between 0 and num_classes-1
word_encoder = LabelEncoder()
word_encoder.fit(train_df['word'])
print('words', len(word_encoder.classes_), '=>', ', '.join([x for x in word_encoder.classes_]))

# Stroke-based Classification
Here we use the stroke information to train a model and see if the strokes give us a better idea of what the shape could be.
A stroke is a line made with the drawing utencil. The stacking function above builds the object by creating a list of strokes made (lines) used to build up the drawing.

In [0]:
def get_Xy(in_df):
    X = np.stack(in_df['drawing'], 0)
    y = to_categorical(word_encoder.transform(in_df['word'].values))
    return X, y
train_X, train_y = get_Xy(train_df)
valid_X, valid_y = get_Xy(valid_df)
test_X, test_y = get_Xy(test_df)
print(train_X.shape)

In [0]:
# plot some of the drawings from the training set along with their labels
fig, m_axs = plt.subplots(3,3, figsize = (16, 16))
rand_idxs = np.random.choice(range(train_X.shape[0]), size = 9)
for c_id, c_ax in zip(rand_idxs, m_axs.flatten()):
    test_arr = train_X[c_id]
    test_arr = test_arr[test_arr[:,2]>0, :] # only keep valid points
    lab_idx = np.cumsum(test_arr[:,2]-1)
    for i in np.unique(lab_idx):
        c_ax.plot(test_arr[lab_idx==i,0], 
                np.max(test_arr[:,1])-test_arr[lab_idx==i,1], '.-')
    c_ax.axis('off')
    c_ax.set_title(word_encoder.classes_[np.argmax(train_y[c_id])])

# LSTM to Parse Strokes
The model suggested from the tutorial is



<img src=https://www.tensorflow.org/images/quickdraw_model.png width="400">

In [0]:
from keras.models import Sequential
from keras.layers import BatchNormalization, Conv1D, LSTM, Dense, Dropout

if len(get_available_gpus())>0:
    print(len(get_available_gpus()), "GPUs are available")
    from keras.layers import CuDNNLSTM as LSTM # this one is about 3x faster on GPU instances

# initialize the sequential model
stroke_read_model = Sequential()

# apply batch normalization to normalize the input layer by adjusting and scaling the activations
stroke_read_model.add(BatchNormalization(input_shape = (None,)+train_X.shape[2:]))

# convolution and dropout layers
stroke_read_model.add(Conv1D(48, (5,)))
stroke_read_model.add(Dropout(0.3))
stroke_read_model.add(Conv1D(64, (5,)))
stroke_read_model.add(Dropout(0.3))
stroke_read_model.add(Conv1D(96, (3,)))
stroke_read_model.add(Dropout(0.3))

# LSTM layers with dropout
stroke_read_model.add(LSTM(128, return_sequences = True))
stroke_read_model.add(Dropout(0.3))
stroke_read_model.add(LSTM(128, return_sequences = False))
stroke_read_model.add(Dropout(0.3))
stroke_read_model.add(Dense(512))
stroke_read_model.add(Dropout(0.3))

# fully connected (dense) layer with softmax activation for prediction
stroke_read_model.add(Dense(len(word_encoder.classes_), activation = 'softmax'))

# load partially pre-trained weights if desired
#stroke_read_model.load_weights(weight_path)

# compile the model using the adam optimizer
stroke_read_model.compile(optimizer = 'adam', 
                          loss = 'categorical_crossentropy', 
                          metrics = ['categorical_accuracy', top_3_accuracy])

# print a summary of the model
stroke_read_model.summary()

In [0]:
# method to save weights after a certain amount of training
checkpoint = ModelCheckpoint(new_weight_path, monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='min', save_weights_only = True)

# method to reduce the learning rate if the loss function plateaus
reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=10, 
                                   verbose=1, mode='auto', min_delta=0.0001, cooldown=5, min_lr=0.0001)

# method to stop training early if the the algorithm does not improve after a certain amount of training
early = EarlyStopping(monitor="val_loss", 
                      mode="min", 
                      patience=5) # probably needs to be more patient, but kaggle time is limited

# place each development above in the model using the callbacks list
callbacks_list = [checkpoint, early, reduceLROnPlat]

In [0]:
# Train the data for n epochs
from IPython.display import clear_output
stroke_read_model.fit(train_X, train_y,
                      validation_data = (valid_X, valid_y), 
                      batch_size = batch_size,
                      epochs = 2,
                      callbacks = callbacks_list)

In [0]:
# load some pretrained weights to see how well the model can do if we train for a while
stroke_read_model.load_weights(weight_path)

# evaluate the pretrained weights
lstm_results = stroke_read_model.evaluate(test_X, test_y, batch_size = batch_size)

print('Accuracy: %2.1f%%, Top 3 Accuracy %2.1f%%' % (100*lstm_results[1], 100*lstm_results[2]))

In [0]:
# test the model showing the precision, recall, and f1-score

# keras has some convenient functions for evaluating the model
from sklearn.metrics import confusion_matrix, classification_report

test_cat = np.argmax(test_y, 1)
pred_y = stroke_read_model.predict(test_X, batch_size = batch_size)
pred_cat = np.argmax(pred_y, 1)

# show the confusoin matrix (just a way to see how well the model did)
plt.matshow(confusion_matrix(test_cat, pred_cat))

print(classification_report(test_cat, pred_cat, 
                            target_names = [x for x in word_encoder.classes_]))

# Reading Point by Point

In [0]:
# show the images point by point and see what the classifier believes the image
# is at each step

points_to_use = [5, 15, 20, 30, 40, 50]
points_to_user = [108]
samples = 12
word_dex = lambda x: word_encoder.classes_[x]
rand_idxs = np.random.choice(range(test_X.shape[0]), size = samples)
fig, m_axs = plt.subplots(len(rand_idxs), len(points_to_use), figsize = (24, samples/8*24))
for c_id, c_axs in zip(rand_idxs, m_axs):
    res_idx = np.argmax(test_y[c_id])
    goal_cat = word_encoder.classes_[res_idx]
    
    for pt_idx, (pts, c_ax) in enumerate(zip(points_to_use, c_axs)):
        test_arr = test_X[c_id, :].copy()
        test_arr[pts:] = 0 # short sequences make CudnnLSTM crash, ugh 
        stroke_pred = stroke_read_model.predict(np.expand_dims(test_arr,0))[0]
        top_10_idx = np.argsort(-1*stroke_pred)[:10]
        top_10_sum = np.sum(stroke_pred[top_10_idx])
        
        test_arr = test_arr[test_arr[:,2]>0, :] # only keep valid points
        lab_idx = np.cumsum(test_arr[:,2]-1)
        for i in np.unique(lab_idx):
            c_ax.plot(test_arr[lab_idx==i,0], 
                    np.max(test_arr[:,1])-test_arr[lab_idx==i,1], # flip y
                      '.-')
        c_ax.axis('off')
        if pt_idx == (len(points_to_use)-1):
            c_ax.set_title('Answer: %s (%2.1f%%) \nPredicted: %s (%2.1f%%)' % (goal_cat, 100*stroke_pred[res_idx]/top_10_sum, word_dex(top_10_idx[0]), 100*stroke_pred[top_10_idx[0]]/top_10_sum))
        else:
            c_ax.set_title('%s (%2.1f%%), %s (%2.1f%%)\nCorrect: (%2.1f%%)' % (word_dex(top_10_idx[0]), 100*stroke_pred[top_10_idx[0]]/top_10_sum, 
                                                                 word_dex(top_10_idx[1]), 100*stroke_pred[top_10_idx[1]]/top_10_sum, 
                                                                 100*stroke_pred[res_idx]/top_10_sum))

# Submission
We can create a submission to Kaggle using the following model

In [0]:
print(test_path)
sub_df = pd.read_csv(test_path)
sub_df['drawing'] = sub_df['drawing'].map(_stack_it)

In [0]:
sub_vec = np.stack(sub_df['drawing'].values, 0)
sub_pred = stroke_read_model.predict(sub_vec, verbose=True, batch_size=batch_size)

In [0]:
top_3_pred = [word_encoder.classes_[np.argsort(-1*c_pred)[:3]] for c_pred in sub_pred]

In [0]:
top_3_pred = [' '.join([col.replace(' ', '_') for col in row]) for row in top_3_pred]
top_3_pred[:3]

# Show some predictions

In [0]:
fig, m_axs = plt.subplots(3,3, figsize = (16, 16))
rand_idxs = np.random.choice(range(sub_vec.shape[0]), size = 9)
for c_id, c_ax in zip(rand_idxs, m_axs.flatten()):
    test_arr = sub_vec[c_id]
    test_arr = test_arr[test_arr[:,2]>0, :] # only keep valid points
    lab_idx = np.cumsum(test_arr[:,2]-1)
    for i in np.unique(lab_idx):
        c_ax.plot(test_arr[lab_idx==i,0], 
                np.max(test_arr[:,1])-test_arr[lab_idx==i,1], '.-')
    c_ax.axis('off')
    c_ax.set_title(top_3_pred[c_id])

In [0]:
inc = 1 # save file increment number
submission_file_path = working_dir+"/submission_"+str(inc)+".csv"

while os.path.isfile(submission_file_path):
  inc = inc + 1
  submission_file_path = working_dir+"/submission_"+str(inc)+".csv"

with open (submission_file_path,'a') as submission_file:
  sub_df['word'] = top_3_pred
  sub_df[['key_id', 'word']].to_csv(submission_file, index=False)

print("Saving as next increment: %d" %inc)
print("Saved file: %s" %"submission_"+str(inc)+".csv")

Things to try:
1. different optimizers
2. increasing the amount of data used 
3. different NN architecture
4. run for more epochs with patience increased